In [1]:
import sys
sys.path

['/Users/noopy/instagram_hashtag_analysis',
 '/Users/noopy/opt/anaconda3/lib/python37.zip',
 '/Users/noopy/opt/anaconda3/lib/python3.7',
 '/Users/noopy/opt/anaconda3/lib/python3.7/lib-dynload',
 '',
 '/Users/noopy/opt/anaconda3/lib/python3.7/site-packages',
 '/Users/noopy/opt/anaconda3/lib/python3.7/site-packages/aeosa',
 '/Users/noopy/opt/anaconda3/lib/python3.7/site-packages/IPython/extensions',
 '/Users/noopy/.ipython']

In [2]:
# kinds_of_exercise = ["자이로토닉","필라테스", "요가","빈야사","발레","키네시스","기구필라테스","매트필라테스","소도구필라테스","플라잉필라테스","헬스","핫요가","플라잉","하타요가","아쉬탕가","인요가","요가쿠아","번지요가","에이리얼후프","발레핏","수영","등산","조깅","크로스핏","사이클","스피닝",
# "타바타","복싱","플라잉요가","파운드핏","드럼스틱","주짓수","번지피트니스","
# 폴댄스","방송댄스","에어로빅","뮤직복싱","번지댄스","줌바","힙레","재즈댄스","살사","바차타","라틴댄스","탱고","펜싱","검도","암벽등반","필록싱","다빈치바디보드","점핑피트니스","트램폴린","볼링","수상스키","서핑","패들보드",\
#                       "아쿠아바이크","패들핏","아쿠아로빅","아쿠아테크","PT"]

In [3]:
# -*- coding: utf-8 -*-
import re
from konlpy.tag import Mecab, Okt
from collections import Counter
import pandas as pd
import numpy as np

def preprocessing_hangul(text):
    # 개행문자 제거
    hangul = re.compile('[^ ㄱ-ㅣ가-힣]+')
    result = hangul.sub('', text)
    return result


In [4]:
useless_list = ['점핑', '동작', '광주', '가능', '사진', '데일리', '공연', '하루', '카페', '영상', '바디', '일산', '검도', '살사', '대구', '사람', '볼링', '동요', '조깅', '등산', '체형', '클럽', '피지', '휘트', '무료', '자이로', '드럼', '발레', '수업', '폴린', '자격증', '피트', '트램', '홍대', '등록', '준비', '수영장', '센터', '댄스', '패들', '토닉', '요가', '일상', '클라이밍', '필라테스', '회원', '잠실', '여행', '취미', '여자', '킥복싱', '번지', '후프', '학원', '교육', '니스', '사랑', '부산', '스튜디오', '피티', '선생', '플라이', '레슨', '남자', '선수', '강사', '친구', '자세', '서핑', '교정', '클래스', '무용', '행복', '그룹', '수련', '트레이닝', '방송', '플라잉', '바이크', '수영', '키즈', '동호회', '지도자', '오늘', '타요', '개인', '헬스', '할인', '에어로빅', '기부', '상담', '라틴', '시작', '맛집', '아쿠아', '다이어트', '파운드', '카톡', '건강', '오픈', '마음', '수중', '탕가', '감사', '진행', '운동', '복싱', '소통', '문의', '전화', '사이클', '서프', '야사', '크로스', '근력', '기구', '시간', '핫요가', '피닝', '과정', '스포츠', '보드']
useful_list = ['점핑', '검도', '살사', '볼링', '조깅', '등산', '체형', '발레', '수영장', '센터', '토닉', '요가' '클라이밍', '필라테스', '취미', '킥복싱', '번지', '후프', '스튜디오', '피티', '플라이', '레슨', '자세', '서핑', '교정', '무용', '수련', '트레이닝', '플라잉', '바이크', '수영', '동호회', '개인', '헬스', '할인', '에어로빅', '아쿠아', '다이어트', '수중', '복싱', '사이클', '크로스', '기구', '핫요가', '보드']
useless_list = [item for item in useless_list if item not in useful_list]
print(useless_list)

['동작', '광주', '가능', '사진', '데일리', '공연', '하루', '카페', '영상', '바디', '일산', '대구', '사람', '동요', '클럽', '피지', '휘트', '무료', '자이로', '드럼', '수업', '폴린', '자격증', '피트', '트램', '홍대', '등록', '준비', '댄스', '패들', '요가', '일상', '클라이밍', '회원', '잠실', '여행', '여자', '학원', '교육', '니스', '사랑', '부산', '선생', '남자', '선수', '강사', '친구', '클래스', '행복', '그룹', '방송', '키즈', '지도자', '오늘', '타요', '기부', '상담', '라틴', '시작', '맛집', '파운드', '카톡', '건강', '오픈', '마음', '탕가', '감사', '진행', '운동', '소통', '문의', '전화', '서프', '야사', '근력', '시간', '피닝', '과정', '스포츠']


In [8]:
def yield_combined_df(exercise_name):
    file_name = "#" + exercise_name + "_sum.txt"
    file_1 = "/Users/noopy/FitCuration/" + file_name
    text = open(file_1, 'r', -1, "UTF-8", errors="ignore").read()
    # text

    clean_text = preprocessing_hangul(text)
    clean_text

    # MeCab으로 명사 뽑아내기
    mecab = Mecab()
    noun_list_mecab = mecab.nouns(clean_text)

    # stopwards preprocessing
    stopwords_mecab = ['수','퀄리티','도시','분','전문','스타','년','원',\
                       '월','화','수','목','금','시','앤','일','그램','문'] 
    stopwords_mecab += useless_list

    clean_noun_list_mecab = []

    for n in noun_list_mecab:
        if n not in stopwords_mecab:
            clean_noun_list_mecab.append(n)

    # get top 100 most common nouns
    nouns_mecab = Counter(clean_noun_list_mecab)
    tags_mecab = nouns_mecab.most_common(100)

    # Okt(Twitter 업데이트 버전)으로 서술어 뽑아내기

    twitter = Okt()

    # 4. 각 문장별로 형태소 구분하기
    morphed_list_okt = twitter.pos(clean_text)
    # morphed_list

    # 불용어
    stopwords_Twitter = ["입니다","있는","있습니다","같은","안녕하세요","고마워요","있어요","있게"\
                         ,"있도록","부탁드립니다","하는","합니다","할","하세요","하기","해","됩니다","하여",'잘','된','되고','되어','되었습니다',"없는","드립니다"\
                        ,"되기","하시는","하고","않을","같다","싶다","이런","저런","그런",'바랍니다'\
                        ,"했습니다","했다","해드립니다","하신","하실","않고","해요","가능합니다","하고싶으신"\
                        ,"않으며","주세요","오세요"]

    # 5. 형용사 품사만 뽑아내기
    adj_list_okt = []
    for word, tag in morphed_list_okt:
        if tag in ['Adjective','Verb'] and word not in stopwords_Twitter:
            adj_list_okt.append(word)

    # 6. 선별된 품사별 빈도수 계산 & 상위 빈도 10위 까지 출력
    adj_counts_okt = Counter(adj_list_okt)
    common_adj_okt = adj_counts_okt.most_common(100)
    # common_adj_okt
    noun_df = pd.DataFrame(np.sort(np.array(tags_mecab), axis=1), columns=[exercise_name+"명사개수",exercise_name+'관련명사'])
    adj_df = pd.DataFrame(np.sort(np.array(common_adj_okt), axis=1), columns=[exercise_name+ "서술어개수",exercise_name+'관련서술어'])
    combined_df = pd.concat([noun_df,adj_df],axis=1)
    return combined_df

In [9]:
# sample_exercise_list = ["자이로토닉","플라잉요가"]
exercise_to_loop = ["자이로토닉","필라테스", "요가","빈야사","키네시스","기구필라테스","매트필라테스","소도구필라테스","플라잉필라테스","헬스","핫요가","플라잉","하타요가","아쉬탕가","인요가","요가쿠아","번지요가","에이리얼후프","발레핏","수영","등산","조깅","크로스핏","사이클","스피닝",
"타바타","복싱","플라잉요가","파운드핏","주짓수","번지피트니스","폴댄스","방송댄스","에어로빅","뮤직복싱","번지댄스","줌바","힙레","재즈댄스","살사","바차타","라틴댄스","탱고","펜싱","검도","암벽등반","필록싱","다빈치바디보드","점핑피트니스","트램폴린","볼링","수상스키","서핑","패들보드",\
                     "아쿠아바이크","패들핏","아쿠아로빅","아쿠아테크","발레"]

exercise_to_loop.sort()
exercise_to_loop

['검도',
 '기구필라테스',
 '다빈치바디보드',
 '등산',
 '라틴댄스',
 '매트필라테스',
 '뮤직복싱',
 '바차타',
 '발레',
 '발레핏',
 '방송댄스',
 '번지댄스',
 '번지요가',
 '번지피트니스',
 '복싱',
 '볼링',
 '빈야사',
 '사이클',
 '살사',
 '서핑',
 '소도구필라테스',
 '수상스키',
 '수영',
 '스피닝',
 '아쉬탕가',
 '아쿠아로빅',
 '아쿠아바이크',
 '아쿠아테크',
 '암벽등반',
 '에어로빅',
 '에이리얼후프',
 '요가',
 '요가쿠아',
 '인요가',
 '자이로토닉',
 '재즈댄스',
 '점핑피트니스',
 '조깅',
 '주짓수',
 '줌바',
 '크로스핏',
 '키네시스',
 '타바타',
 '탱고',
 '트램폴린',
 '파운드핏',
 '패들보드',
 '패들핏',
 '펜싱',
 '폴댄스',
 '플라잉',
 '플라잉요가',
 '플라잉필라테스',
 '필라테스',
 '필록싱',
 '하타요가',
 '핫요가',
 '헬스',
 '힙레']

In [10]:
df0 = yield_combined_df('PT')

for item in exercise_to_loop:
    df0 = pd.concat([df0,yield_combined_df(item)],axis=1)
df0.to_csv("combined_nogada.csv", index=False, header=True)